### K-means clustering: text data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision = 3)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# This function returns:
# - a matrix X with one row per document (review). Each row is a sparse
# vector containing tf.idf term weights for the words in the document.
# - the vectorizor used to create X
# - the actual reviews used as input to the vectorizer

def get_beer_reviews_vectorized(top_n = -1, ngram_range = (1,1), max_features = 1000):
    df = pd.read_csv('./assets/beer2.csv')['text']
    df = df.dropna()   # drop any rows with empty reviews
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=max_features,
                                 min_df=2, stop_words='english',
                                 ngram_range = ngram_range,
                                 use_idf=True)
    if (top_n >= 0):
        review_instances = df.values[0:top_n]
    else:
        review_instances = df.values
    
    X = vectorizer.fit_transform(review_instances) 
    
    return (X, vectorizer, review_instances)

def print_cluster_features(vectorizer, centroids, n_clusters, top_n_features):
    terms = vectorizer.get_feature_names()
    for i in range(n_clusters):
        print("Cluster %d:" % i, end='')
        for ind in centroids[i, :top_n_features]:
            print(' [%s]' % terms[ind], end='')
        print()

In [3]:
# Analyze the first 5000 entries (reviews)

(X, vectorizer, review_instances) = get_beer_reviews_vectorized(5000, (1,2))

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
scores = []
for i in range(2,10):
    kmeans = KMeans(n_clusters = i, init='k-means++', max_iter=100, n_init=1, random_state=42)
    model = kmeans.fit_predict(X)
    db_score = davies_bouldin_score(X.toarray(), model)
    model = kmeans.fit(X)
    labels = model.labels_
    ch_score = calinski_harabasz_score(X.toarray(), labels)
    scores.append([db_score,ch_score])
scores

[[7.103724957438545, 66.36912593606624],
 [9.35040755232012, 51.20285111664583],
 [9.592438827030367, 40.613123398634784],
 [8.197282591441104, 42.95571301600999],
 [8.287681685515773, 38.14493770875936],
 [7.976206385944393, 35.09722200514752],
 [7.955940974835473, 31.8349341264373],
 [7.743336777219178, 29.53090074292199]]

In [5]:
for k_value in range(len(scores)):
    print("k-means: ", k_value+2)
    print("Davies-Bouldin Score: ", round(scores[k_value][0],2))
    print("Calinski-Harabsz Score: ", round(scores[k_value][1],2))
    print("\n")


k-means:  2
Davies-Bouldin Score:  7.1
Calinski-Harabsz Score:  66.37


k-means:  3
Davies-Bouldin Score:  9.35
Calinski-Harabsz Score:  51.2


k-means:  4
Davies-Bouldin Score:  9.59
Calinski-Harabsz Score:  40.61


k-means:  5
Davies-Bouldin Score:  8.2
Calinski-Harabsz Score:  42.96


k-means:  6
Davies-Bouldin Score:  8.29
Calinski-Harabsz Score:  38.14


k-means:  7
Davies-Bouldin Score:  7.98
Calinski-Harabsz Score:  35.1


k-means:  8
Davies-Bouldin Score:  7.96
Calinski-Harabsz Score:  31.83


k-means:  9
Davies-Bouldin Score:  7.74
Calinski-Harabsz Score:  29.53




The two evaluation metrics we will be using for this K-means algorithm are the Davies-Bouldin score, and the Calinski-Harabsz score. The value **k = 2** minimizes Davies-Bouldin score while maximizing the Calinski-Harabsz score.

Now that we have chosen a value for k, we will find what the predominant terms are in the resulting clusters.

In [6]:
kmeans = KMeans(n_clusters = 2, init='k-means++', max_iter=100, n_init=1, random_state=42)
kmeans.fit(X)
centroid =  kmeans.cluster_centers_
sort_centroid = np.argsort(-1*centroid)
print_cluster_features(vectorizer, sort_centroid, n_clusters =2, top_n_features = 10)

Cluster 0: [chocolate] [coffee] [dark] [roasted] [black] [brown] [stout] [nice] [like] [malt]
Cluster 1: [nice] [hops] [good] [light] [sweet] [malt] [like] [bit] [hop] [white]


##### The words above were simply the words in a typical review for each cluster, but some words such as "like" and "nice" appear in both clusters. It would be more informative to find words that are specific (or discriminative) to clusters

Information gain (IG) helps find features that distinguish one class from another, improving prediction accuracy. We don't have labels, but we use the same idea to find terms that are special to a cluster, this distinguishing that cluster from another. 

In [7]:
from scipy.stats import entropy

def compute_distinctive_term_score(T, T_a):   
    # First compute information gain.
    IG = entropy(T) - entropy(T_a) 
          
    # if it's high IG, but not for this class, we want to penalize,
    # so flip the IG negative.  We do this because these are terms those whose *absence* is notable, 
    # we give them a score that guarantees we won't rank them highly.
    if (T_a[0] < T_a[1]):
        score = -IG  
    else:
        score = IG
    return score

# create a 1-vs-all two-class matrix for each cluster
def one_vs_all_count_matrix(m, index):
    # row zero is the selected row
    row0 = m[index, :]
    # row one is the other rows summed
    row1 = np.vstack((m[0:index, :], m[index+1:, :])).sum(axis=0)
              
    result = np.vstack((row0, row1))
    return result

#### Here are the steps I followed to reach information gain based distinctive terms:
1. Run k-means again but this time setting k=7 to give a slightly more diverse set of clusters to start with.


2. Get the cluster-term matrix of cluster centers $L$ from the result of k-means, an array with seven rows, one row per cluster, and 1000 columns (one column per word/term in the vocabulary produced by the vectorizer).


3. For each row $c$ in $L$,  (for each cluster)

    a. For this cluster, we're going to compute a 'distinctive term score' for each of the terms/words in the vocabulary. To do this, we're going to compute how surprised we are at the distribution $T_w$ of a word $w$ between this cluster vs all other clusters, compared to the distribution $T_c$ of the average word between this cluster $c$ and all clusters. If a word is a lot more likely to occur in this cluster than we would expect by chance, it is considered a more distinctive word for this cluster. This surprise factor is *information gain*. 
    
    Te variable $T_c$ has two possible values: "in this cluster $c$" and "in any other cluster". Each of these outcomes has a probability when we're talking about where words occur.
    
    The function `one_vs_all_count_matrix` makes it easy to compute $T_c$. As input, you provide the matrix $L$ along with the row $c$ you want to analyze. The output is a "one versus all" matrix $M$ that has shape (2, 1000): the first row has the weights of words in *this* cluster $c$ (which is the same as L[c, :]), and the second row has the aggregate summed weights of the words across all *other* clusters. 
    
    b. To compute $T_c$, compute $M$ and sum over all columns. Then to turn it into a probability distribution, normalize by computing `T_c = T_c / sum(T_c)`.
    
    c. Now we loop through each column $i$ of $L$. We'll call the term corresponding to the $i$-th column, term $w$. So for each term $w$ we'll compute the second part we need: the term-specific distribution $T_w$ for that specific term $w$ in this cluster vs all clusters. 
    
    $T_w$ is the $i$-th column of the one-vs-all matrix $M$ (term weight in this cluster, term weight in all other clusters)  again normalized by computing 'T_w = T_w / sum(T_w)'. 
    
    Then use the `compute_distinctive_term_score` function to compute the information gain for that term. The first argument is $T_c$ computed for all words, and the second argument is $T_w$ just for this term $w$. 
    
    d. After looping through all terms in (c) above, we have a result list with 1000 entries, one per term, containing the distinctive score for each term, for that cluster $c$.
    
    e. Get the most distinctive terms for this cluster $c$ by sorting the array by descending score and selecting the top.

In [8]:
(X, vectorizer, review_instances) = get_beer_reviews_vectorized(5000, (1,2))

def cluster_features(vectorizer, centroids, n_clusters, top_n_features):
    terms = vectorizer.get_feature_names()
    final_list = []
    for i in range(n_clusters):
        cluster_list = []
        for ind in centroids[i, :top_n_features]:
            cluster_list.append(terms[ind])
        final_list.append(cluster_list)
    return final_list

def cluster_labeling():
    # Step 1
    kmeans = KMeans(n_clusters = 7, init='k-means++', max_iter=100, n_init=1, random_state=42)
    kmeans.fit(X)
    
    # Step 2
    L =  kmeans.cluster_centers_
    result_list = []
    
    # Step 3
    for k in range(7):
        # Step 3a
        M = one_vs_all_count_matrix(L,k)
        
        # Step 3b
        T_c = np.sum(M, axis=1)
        T_c = T_c / sum(T_c)
        
        # Step 3c
        for i in range(1000):
            T_w = M[:,i]
            T_w = T_w / sum(T_w)
            
            # Step 3d
            result_list.append(compute_distinctive_term_score(T_c, T_w))
            
    # Step 3e
    result_reshape =np.reshape(result_list, (7,1000))
    sort_result = np.argsort(-1*np.array(result_reshape))   
    result = cluster_features(vectorizer, sort_result, n_clusters =7, top_n_features = 5)
    return result
cluster_labeling()


[['vintage', 'sugar', 'toffee', 'nuts', 'barleywine'],
 ['pumpkin pie', 'pumpkin ale', 'pumpkin beer', 'pumpkin', 'orange color'],
 ['woody', 'apple', 'wood', 'peach', 'apples'],
 ['pale malt', 'light bodied', 'inch head', 'grain', 'pils'],
 ['caramel malt', 'sticky lacing', 'bitter hops', 'hop', 'hop bite'],
 ['marzen', 'bar', 'went', 'sam', 'oktoberfest'],
 ['roasted coffee', 'opaque', 'imperial', 'dark brown', 'colored head']]